<a href="https://colab.research.google.com/github/Mo-Bkny/python/blob/master/UMUZI_Advance_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Helper function to help you load the data

def load_data(data_dir):
    images = []
    labels = []

    # Iterate through each fracture type directory
    for fracture_type in os.listdir(data_dir):
        fracture_dir = os.path.join(data_dir, fracture_type)

        # Check if it's a directory
        if os.path.isdir(fracture_dir):
            # Load training images
            train_dir = os.path.join(fracture_dir, 'Train')
            for img_name in os.listdir(train_dir):
                img_path = os.path.join(train_dir, img_name)
                img = Image.open(img_path).convert('RGB')  # Ensure images are in RGB format
                img = img.resize((224, 224))  # Resize if needed
                images.append(np.array(img))
                labels.append(fracture_type)  # Use the directory name as the label

            # Load testing images
            test_dir = os.path.join(fracture_dir, 'Test')
            for img_name in os.listdir(test_dir):
                img_path = os.path.join(test_dir, img_name)
                img = Image.open(img_path).convert('RGB')
                img = img.resize((224, 224))
                images.append(np.array(img))
                labels.append(fracture_type)

    return np.array(images), np.array(labels)

# Example usage
data_directory = '/content/drive/MyDrive/Bone Break Classification/' # Change appropriately
images, labels = load_data(data_directory)

print(f'Loaded {len(images)} images with {len(set(labels))} unique classes.')

Loaded 1129 images with 10 unique classes.
